In [1]:
import os
import glob
import sqlite3
from enum import Enum
from sqlite3 import Error

In [2]:
class DocumentClass(Enum):
    ADMISIONES = 1
    DEMANDAS_SELLADAS_EJECUCION = 2
    DEMANDAS_SELLADAS_MONITORIO = 3
    FIN_DE_MONITORIO = 4
    HIPOTECARIO_PLAZO = 5
    HIPOTECARIO_TRAMITE = 6
    INADMISIONES = 7
    MANDAMIENTO = 8

In [3]:
pathToDB = r'C:\TEMP\mails2.db'

sqlTableCreationStr = """ CREATE TABLE "Mails" (
                       "id" INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
                       "mail_name" TEXT NOT NULL,
                       "mail_body" TEXT NOT NULL,
                       "attachment_name" TEXT NOT NULL,
                       "attachment_body" TEXT NOT NULL,
                       "mail_type" TEXT NOT NULL
                       );"""

sqlMailInsertStr = """ INSERT INTO Mails(mail_name, mail_body, attachment_name, attachment_body, mail_type)
                      VALUES (?, ?, ?, ?, ?) """

def MakeConnectionToDB(_pathToDBfile):
    try:
        _connection = sqlite3.connect(_pathToDBfile)
        if _connection is None:
            print('Error! cannot create the database connection.')
            return None
        return _connection
    except Error as _e:
        print(_e)
    return None

def CreateTable(_connection, _sqlTableCreationStr):
    try:
        _cursor = _connection.cursor()
        if _cursor is None:
            print('Error! cannot create the database cursor.')
            return None
        _cursor.execute(_sqlTableCreationStr)
        _connection.commit()
    except Error as _e:
        print(_e)

def InsertMail(_connection, _sqlMailInsertStr, _sqlMailValuesStr):
    try:
        _cursor = _connection.cursor()
        if _cursor is None:
            print('Error! cannot create the database cursor.')
            return None
        _cursor.execute(_sqlMailInsertStr, _sqlMailValuesStr)
        _connection.commit()
        return _cursor.lastrowid
    except Error as _e:
        print(_e)
    return None

def ExecuteSQL(_connection, _sqlStr):
    try:
        _cursor = _connection.cursor()
        if _cursor is None:
            print('Error! cannot create the database cursor.')
            return None
        _cursor.execute(_sqlStr)
        _connection.commit()
    except Error as _e:
        print(_e)
    return None

def GetAllMailsFromTable(_connection, _sqlTableStr):
    try:
        _cursor = _connection.cursor()
        if _cursor is None:
            print('Error! cannot create the database cursor.')
            return None
        _cursor.execute('SELECT * FROM ' + _sqlTableStr)
        return _cursor.fetchall()
    except Error as _e:
        print(_e)
    return None

def CloseConnection(_connection):
    _connection.close()

def readTextFile(path):
    with open(path, 'r', encoding='utf-8', errors='ignore') as file:
        return file.read()

In [4]:
c = MakeConnectionToDB(pathToDB)
with c:
    CreateTable(c, sqlTableCreationStr)

In [5]:
c = MakeConnectionToDB(pathToDB)
with c:
    PathToFolders = r'C:\\TEMP'
    for docClass in DocumentClass:
        currentFolder = os.path.join(PathToFolders, str(docClass.value))
        mailType = docClass.name                                                                               # mail_type
        mailsFolder = os.listdir(currentFolder)
        for i in range(len(mailsFolder)):
            currentMailFolder = os.path.join(currentFolder, mailsFolder[i])
            mailBodyFileName = glob.glob(currentMailFolder + '\\\\*.msg.txt')[0]
            attachmentDirName = 'a'
            attachmentDir = os.path.join(currentMailFolder, attachmentDirName)
            mailBodyFile = os.path.join(currentMailFolder, mailBodyFileName)
            mailName = os.path.basename(mailBodyFile)[:-4]                                                     # mail_name
            mailBody = readTextFile(mailBodyFile)                                                              # mail_body
            attachments = os.listdir(attachmentDir)
            for j in range(len(attachments)):
                currentAttachment = os.path.join(attachmentDir, attachments[j])
                attachmentName = attachments[j]                                                                # attachment_name
                attachmentBody = readTextFile(currentAttachment)                                               # attachment_body
                InsertMail(c, sqlMailInsertStr, (mailName, mailBody, attachmentName, attachmentBody, mailType))

In [ ]:
c = MakeConnectionToDB(pathToDB)
with c:
    rows = GetAllMailsFromTable(c, "Mails")
    print(len(rows))
    for row in rows[:20]:
        print(row)